In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import calendar
from datetime import datetime
import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
path = os.getcwd()

# Scraping of Equity Data

In [4]:
def download_equity():

    path = os.path.join(os.getcwd(),"Data")
    security_url = "https://www.bseindia.com/corporates/List_Scrips.aspx"

    if os.path.exists(os.path.join(path,"Equity.csv")):
        print("Equity.csv exists")
        return

    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--headless")
    chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
    driver = webdriver.Chrome(ChromeDriverManager().install(),options = chromeOptions)
    driver.get(security_url)
    
    # to select Equity
    equity = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
    equity = Select(equity)
    equity.select_by_visible_text("Equity")  

    # to select Active
    active = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
    active = Select(active)
    active.select_by_visible_text("Active") 
    
    # to click submit 
    submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
    submit.send_keys(Keys.RETURN)

    # to download csv file
    driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i").click()
    time.sleep(3)
    driver.quit()

# Scraping of stock data

In [5]:
def download_stocks(security_id):

    path = os.path.join(os.getcwd(),"Data\\Stock")
    stock_url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"

    def convert_date_to_unix_timestamp(stock_df):
        stock_df["Unix Date"] = stock_df["Date"].apply(lambda x : time.mktime(x.timetuple()))
        return stock_df

    # set from date
    def set_from_date(d,m,y):
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
        from_date.clear()
        from_date.click()
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    # set to date
    def set_to_date(d,m,y):
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
        to_date.clear()
        to_date.click()
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()
    

    def set_security_id(security):
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)

    def download():
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[2]/div/div[1]/div[2]/span/a/i").click()
        time.sleep(6)
        driver.quit()

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    if os.path.exists(os.path.join(path,str(security_id)+".csv")):
        driver = create_driver()
        driver.get(stock_url)
        old_df = pd.read_csv(os.path.join(path,str(security_id)+".csv"))
        old_df["Date"] = pd.to_datetime(old_df["Date"])
        last = old_df["Date"].head(1)[0]
        set_security_id(str(security_id))

        set_from_date(last.day,calendar.month_abbr[last.month],str(last.year))
        today = datetime.date.today()
        # today = last+datetime.timedelta(365)
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()
        new_df = pd.read_csv(os.path.join(path,str(security_id)+" (1).csv"))
        new_df["Date"] = pd.to_datetime(new_df["Date"],errors="coerce")
        new_df = new_df.drop(columns = ["Unnamed: 13"],axis=1,errors='ignore')
        new_df = new_df.dropna(how='all')
        new_df = convert_date_to_unix_timestamp(new_df)
        res = new_df.append(old_df)
        res.to_csv(os.path.join(path,str(security_id)+".csv"),index=None)
        os.remove(os.path.join(path,str(security_id)+" (1).csv"))
        return new_df

    else:
        driver = create_driver()
        driver.get(stock_url)
        set_security_id(str(security_id))
        set_from_date("02","Aug","2007")
        today = datetime.date.today()
        # start = datetime.datetime.strptime("01 Jan 2000","%d %b %Y")
        # today = start+datetime.timedelta(365)
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()
        stock =  pd.read_csv(os.path.join(path,str(security_id)+".csv"))
        stock.Date = pd.to_datetime(stock.Date,errors="coerce")
        stock = stock.drop(columns = ["Unnamed: 13"],axis=1,errors='ignore')
        stock = stock.dropna(how='all')
        stock = convert_date_to_unix_timestamp(stock)
        stock.to_csv(os.path.join(path,str(security_id)+".csv"),index=None)
        return stock

# Scraping of Corporate Actions Data

In [6]:
def download_corporate_actions(security_id):

    path = os.path.join(os.getcwd(),"Data\\CorporateActions")
    corporate_url = "https://www.bseindia.com/corporates/corporate_act.aspx"

    # set from date
    def set_from_date(d,m,y):
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
        from_date.clear()
        from_date.click()
        year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    # set to date
    def set_to_date(d,m,y):
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
        to_date.clear()
        to_date.click()
        year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            print(year.options[0].text,y)
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    def set_security_id(security):
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_SmartSearch_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
    

    def download():
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i").click()
        time.sleep(3)
        driver.quit()

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    driver = create_driver()
    driver.get(corporate_url)
    set_security_id(str(security_id))
    set_from_date("01","Jan","1991")
    today = datetime.date.today()
    set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
    download()
    if os.path.exists(os.path.join(path,str(security_id)+".csv")):
        os.remove(os.path.join(path,str(security_id)+".csv"))
    os.rename(os.path.join(path,"Corporate_Actions.csv"),os.path.join(path,str(security_id)+".csv"))
    

# Scraping of Index Data

In [7]:
def download_index():

    
    index_url = "https://www.bseindia.com/indices/IndexArchiveData.html"
    index = "S&P BSE 500"
    path = os.path.join(os.getcwd(),"Data")

    # set from date
    def set_from_date(d,m,y):
        from_date = driver.find_element_by_xpath('//*[@id="txtFromDt"]')
        from_date.clear()
        from_date.click()
        year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    # set to date
    def set_to_date(d,m,y):
        to_date = driver.find_element_by_xpath('//*[@id="txtToDt"]')
        to_date.clear()
        to_date.click()
        year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            print(year.options[0].text,y)
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()


    def set_index(index_):
        indexes = driver.find_element_by_xpath('//*[@id="ddlIndex"]')
        indexes = Select(indexes)
        indexes.select_by_visible_text(index_)  

    def download():
        submit = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[5]/div/input')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/i").click()
        time.sleep(3)
        driver.quit()

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    if os.path.exists(os.path.join(path,"Index.csv")):
        old_df = pd.read_csv(os.path.join(path,"Index.csv"),names=["Date","Open","High","Low","Close"])
        old_df = old_df.iloc[1:]
        old_df["Date"] = pd.to_datetime(old_df["Date"])
        last = old_df["Date"].tail(1)[old_df.shape[0]]

        driver = create_driver()
        driver.get(index_url)
        set_index("S&P BSE 500")
        set_from_date(last.day,calendar.month_abbr[last.month],str(last.year))
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()

        new_df = pd.read_csv(os.path.join(path,"CSVForDate.csv"),names=["Date","Open","High","Low","Close"])
        new_df = new_df.iloc[1:]
        new_df["Date"] = pd.to_datetime(new_df["Date"])
        res = old_df.append(new_df)
        os.remove(os.path.join(path,"Index.csv"))
        res.to_csv(os.path.join(path,"Index.csv"),index=None)
        os.remove(os.path.join(path,"CSVForDate.csv"))
    else:
        driver = create_driver()
        driver.get(index_url)
        set_index("S&P BSE 500")
        set_from_date("2","Aug","2007")
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()
        res = pd.read_csv(os.path.join(path,"CSVForDate.csv"),names=["Date","Open","High","Low","Close"])
        res = res.iloc[1:]
        res["Date"] = pd.to_datetime(res["Date"])
        os.remove(os.path.join(path,"CSVForDate.csv"))
        res.to_csv(os.path.join(path,"Index.csv"),index=None)

# Scraping Risk Free Rate Data

In [8]:

def download_risk_free_rate():

    path = os.path.join(os.getcwd(),"Data")
    risk_free_rate_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    def download():
        ele = driver.find_element_by_xpath('//*[@id="interestRateTimePeriod"]')
        ele = Select(ele)
        ele.select_by_visible_text("All") 

        btn = driver.find_element_by_xpath('/html/body/form/div[8]/div/div[1]/div/div[2]/div/div/div/div[1]/div[2]/div/table/tbody/tr/td/div/div[3]/div[2]/input')
        btn.click()
        time.sleep(15)
        soup = BeautifulSoup(driver.page_source,'html')
        driver.quit()
        table = soup.find_all("table", {"class": "t-chart"})
        risk_free_rate = pd.read_html(str(table))[0]
        # risk_free_rate.to_csv(os.path.join(path,"inRiskFreeRate.csv"),index=None)
        risk_free = risk_free_rate[["Date","3 mo"]]
        risk_free_rate["Rate"] = risk_free_rate["3 mo"]
        risk_free.columns = ["Date","Rate"]
        risk_free.dropna(inplace=True)
        risk_free.to_csv(os.path.join(path,"RiskFreeRate.csv"),index=None)
    driver = create_driver()
    driver.get(risk_free_rate_url)
    download()

# Scraping of Revenue Profit Data

In [9]:
def download_revenue_profit(code,name): 

    path = os.path.join(os.getcwd(),"Data\\Revenue")

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    def download():
        columns = ["security code","security name",'revenue','income','expenditure','profit','eps',"year","quartile"]
        code_df = pd.DataFrame(columns=columns)
        for q in range(55,108):
            url = "https://www.bseindia.com/corporates/results.aspx?Code=" + str(code) +"&Company="+ str(name) +"&qtr="+ str(q) +"&RType=D"
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, "html")

            table = soup.find_all("table",attrs={"id":"ContentPlaceHolder1_tbl_typeID"})
            table = pd.read_html(str(table))[0]
            table = table[[0,1]]
            table.dropna(inplace=True)
            table = table.transpose()
            table.columns = table.iloc[0]
            table = table[1:]
            table.columns = map(str.lower, table.columns)
            table.drop(["description"],inplace=True,axis=1)
            try:
                table["date begin"] = pd.to_datetime(table["date begin"])
                date = table.iloc[0]["date begin"]
                table["quartile"] =  (date.month-1)//3 + 1
                table["year"] = date.year
                table["security name"] = name
                table["security code"] = code
                cols = table.columns
                mycols = ['sales','income','expenditure','profit','eps']
                row = {}
                row["security name"] = name
                row["security code"] = code
                row["year"] = date.year
                row["quartile"] = (date.month-1)//3 + 1

                for my in mycols:
                    res = [c for c in cols if my in c]
                    if my == "income":
                        res = [c for c in res if "total income" == c]
                    elif my == "profit":
                        res = [c for c in res if "net profit" == c]
                    elif my == "expenditure":
                        pass
                    elif my =="eps":
                        pass
                    elif my == "sales":
                        row["revenue"] = table[res].values[0][0]
                        continue
                        pass

                    row[my] = table[res].values[0][0]
                code_df = code_df.append(row,ignore_index=True)
            except Exception as e:
                pass
        code_df.to_csv(os.path.join(path,str(code)+".csv"),index=None)
    driver = create_driver()
    download()

In [ ]:
start = time.time()

In [ ]:
download_equity()

In [ ]:
security_id = "500325"
name = "RELIANCE INDUSTRIES LTD"

In [ ]:
stock_df = download_stocks(security_id)

In [ ]:
download_index()

In [ ]:
download_risk_free_rate()

In [ ]:
download_corporate_actions(security_id)

In [ ]:
download_revenue_profit(security_id,name)

In [ ]:
end = time.time()

In [ ]:
print("number of seconds elapsed for downloading top one stocks and its relevant data = ", end-start)

In [10]:
top = pd.read_csv("top.csv")

In [ ]:
start = time.time()

In [12]:
%%time
for _,row in top.iterrows():
    security_id = row["security id"]
    name = row["name"]
    print(security_id,name)
    try:
        download_stocks(security_id)
        download_revenue_profit(security_id,name)
        download_corporate_actions(security_id)
    except Exception as e:
        print(e)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
500112 SBIN
 
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
500325 RELIANCE INDUSTRIES LTD
 
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST

In [13]:
end = time.time()

In [14]:
print("number of seconds elapsed for downloading top 10 stocks and its relevant data = ", end-start)

number of seconds elapsed for downloading top 10 stocks and its relevant data =  795.5786244869232


In [15]:
for _,row in top.iterrows():
    security_id = row["security id"]
    name = row["name"]
    print("Revenue path =",security_id,os.path.exists(os.path.join(path,"Data/Revenue/"+str(security_id)+".csv")))
    print("Stock path =",security_id,os.path.exists(os.path.join(path,"Data/Stock/"+str(security_id)+".csv")))
    print("Corporate path =",security_id,os.path.exists(os.path.join(path,"Data/CorporateActions/"+str(security_id)+".csv")))

Revenue path = 500112 True
Stock path = 500112 True
Corporate path = 500112 True
Revenue path = 500325 True
Stock path = 500325 True
Corporate path = 500325 True
Revenue path = 532540 True
Stock path = 532540 True
Corporate path = 532540 True
Revenue path = 500209 True
Stock path = 500209 True
Corporate path = 500209 True
Revenue path = 532174 True
Stock path = 532174 True
Corporate path = 532174 True
Revenue path = 507685 True
Stock path = 507685 True
Corporate path = 507685 True
Revenue path = 530965 True
Stock path = 530965 True
Corporate path = 530965 True
Revenue path = 500182 True
Stock path = 500182 True
Corporate path = 500182 True
Revenue path = 532210 True
Stock path = 532210 True
Corporate path = 532210 True
Revenue path = 500180 True
Stock path = 500180 True
Corporate path = 500180 True
